<a href="https://colab.research.google.com/github/scharnk/Linear-Classifiers-in-Python/blob/master/CH03_logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularized Logistic Regression

in scikit learn the hyperparameter 'C' is the inverse of the regularization strength
* larger C means less regularization
* smaller C means more
<br><br>
* regularized loss = original loss + large coefficient penalty
* **more regularization: lower training accuracy**
* **more regularization: (almost always) higher test accuracy**
<br><br>
* if large features were causing overfitting, regularization causes less overfitting from diminishing feature size

# L1 vs L2 regularization

Lasso = linear regression with L1 regularization <br>
Ridge = linear regression with L2 regularization <br>
For other models like logistic regression we just say L1, L2, etc.

In [0]:
# Train and validaton errors initialized as empty list
train_errs = list()
valid_errs = list()

# Loop over values of C_value
for C_value in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    # Create LogisticRegression object and fit
    lr = LogisticRegression(C=C_value)
    lr.fit(X_train, y_train)
    
    # Evaluate error rates and append to lists
    train_errs.append( 1.0 - lr.score(X_train, y_train) )
    valid_errs.append( 1.0 - lr.score(X_valid, y_valid) )
    
# Plot results
plt.semilogx(C_values, train_errs, C_values, valid_errs)
plt.legend(("train", "validation"))
plt.show()

# Logistic regression and feature selection

In [0]:
# Specify L1 regularization
lr = LogisticRegression(penalty='l1')

# Instantiate the GridSearchCV object and run the search
searcher = GridSearchCV(lr, {'C':[0.001, 0.01, 0.1, 1, 10]})
searcher.fit(X_train, y_train)

# Report the best parameters
print("Best CV params", searcher.best_params_)

# Find the number of nonzero coefficients (selected features)
best_lr = searcher.best_estimator_
coefs = best_lr.coef_
print("Total number of features:", coefs.size)
print("Number of selected features:", np.count_nonzero(coefs))

In [0]:
# Get the indices of the sorted cofficients
inds_ascending = np.argsort(lr.coef_.flatten()) 
inds_descending = inds_ascending[::-1]

# Print the most positive words
print("Most positive words: ", end="")
for i in range(5):
    print(vocab[inds_descending[i]], end=", ")
print("\n")

# Print most negative words
print("Most negative words: ", end="")
for i in range(5):
    print(vocab[inds_ascending[i]], end=", ")
print("\n")

How are these probabilities computed?
* logistic regression predictions: sign of raw model output
* logistic regression probabilities: "squashed" raw model output
* **USE SIGMOID FUNCTION** 

In [0]:
# Set the regularization strength
model = LogisticRegression(C=____)

# Fit and plot
model.fit(X,y)
plot_classifier(X,y,model,proba=True)

# Predict probabilities on training points
prob = model.predict_proba(X)
print("Maximum predicted probability", np.max(prob))

In [0]:
# Set the regularization strength
model = LogisticRegression(C=0.1)

# Fit and plot
model.fit(X,y)
plot_classifier(X,y,model,proba=True)

# Predict probabilities on training points
prob = model.predict_proba(X)
print("Maximum predicted probability", np.max(prob))

As you probably noticed, smaller values of C lead to less confident predictions. That's because smaller C means more regularization, which in turn means smaller coefficients, which means raw model outputs closer to zero and, thus, probabilities closer to 0.5 after the raw model output is squashed through the sigmoid function.

In [0]:
lr = LogisticRegression()
lr.fit(X,y)

# Get predicted probabilities
proba = lr.predict_proba(X)

# Sort the example indices by their maximum probability
proba_inds = np.argsort(np.max(proba,axis=1))

# Show the most confident (least ambiguous) digit
show_digit(proba_inds[-1], lr)

# Show the least confident (most ambiguous) digit
show_digit(proba_inds[0], lr)

Combining binary classifiers with one-vs-rest
* one-vs-rest is **default** for scikit learn's logistic regression

# One-vs-rest:

* fit a binary classifier for each class
* predict with all, take largest output
* pro: simple, modular
* con: not directly optimizing accuracy
* common for SVMs as well
* can produce probabilities

# "Multinomial" or "softmax":

* fit a single classifier for all classes
* prediction directly outputs best class
* con: more complicated, new code
* pro: tackle the problem directly
* possible for SVMs, but less common
* can produce probabilities

# Model coefficients for multi-class

require non-default solver such as "lbfgs"<br>
example below

In [0]:
In [1]: lr_ovr = LogisticRegression() # one-vs-rest by default

In [2]: lr_ovr.fit(X,y)

In [3]: lr_ovr.coef_.shape
Out[3]: (3,13)

In [4]: lr_ovr.intercept_.shape
Out[4]: (3,)
 
In [5]: lr_mn = LogisticRegression(multi_class="multinomial",solver="lbfgs")

In [6]: lr_mn.fit(X,y)

In [7]: lr_mn.coef_.shape
Out[7]: (3,13)

In [8]: lr_mn.intercept_.shape
Out[8]: (3,)

In [0]:
# Fit one-vs-rest logistic regression classifier
lr_ovr = LogisticRegression()
lr_ovr.fit(X_train, y_train)

print("OVR training accuracy:", lr_ovr.score(X_train, y_train))
print("OVR test accuracy    :", lr_ovr.score(X_test, y_test))

# Fit softmax classifier
lr_mn = lr_mn = LogisticRegression(multi_class="multinomial",solver="lbfgs")
lr_mn.fit(X_train, y_train)

print("Softmax training accuracy:", lr_mn.score(X_train, y_train))
print("Softmax test accuracy    :", lr_mn.score(X_test, y_test))

In [0]:
# Print training accuracies
print("Softmax     training accuracy:", lr_mn.score(X_train, y_train))
print("One-vs-rest training accuracy:", lr_ovr.score(X_train, y_train))

# Create the binary classifier (class 1 vs. rest)
lr_class_1 = LogisticRegression(C=100)
lr_class_1.fit(X_train, y_train==1)

# Plot the binary classifier (class 1 vs. rest)
plot_classifier(X_train, y_train==1, lr_class_1)

As you can see, the binary classifier incorrectly labels almost all points in class 1 (shown as red triangles in the final plot)! Thus, this classifier is not a very effective component of the one-vs-rest classifier. In general, though, one-vs-rest often works well.

In [0]:
# We'll use SVC instead of LinearSVC from now on
from sklearn.svm import SVC

# Create/plot the binary classifier (class 1 vs. rest)
svm_class_1 = SVC()
svm_class_1.fit(X_train, y_train==1)
plot_classifier(X_train, y_train==1, svm_class_1)
